# Publish/Subscribe

* **Enviar a mesma mensagem para múltiplos consumidores!**

**Problema**
* Vamos simular um sistema de logs, que publica mensagens para quem quiser consumir. 
* Estamos interessados somente em mensagens atuais, sem nos importar com eventos passados.

**Perguntas**

Dado o esquema genérico de criação, envio e consumo de mensagens e os códigos produzidos:

![prefetch-count](https://s3-sa-east-1.amazonaws.com/lcpi/7cfc070b-dd79-4ec5-8d28-e15bd745e37f.png)


1.  Explique por que o código anterior não satisfaz nosso requerimento atual.
2.  Como você espera que o diagrama seja modificado?

## Exchange

Até agora, por motivos pedagógicos, entregamos mensagens diretamente para uma fila, porém, com frequência, o produtor não sabe da existência de filas específicas. 





O produtor deve entregar as mensagens para um *exchange*. O *exchange* recebe mensagens de um produtor e as publica nas filas. Ele deve saber exatamente o que fazer com e para onde mandar cada mensagem!

As regras que definem o que fazer com cada mensagem são definidas pelo *exchange_type*. Os quatro tipos disponíveis são:
* direct
* topic
* headers
* fanout

![exchanges](https://s3-sa-east-1.amazonaws.com/lcpi/75dde901-20c9-49ee-ba81-f75f7ee04c57.png)

### fanout exchange


* Encaminha todas as mensagens que recebe para todas as filas que conhece!

Para criarmos um exchange com nome `logs` do tipo `fanout` e publicarmos mensagens nele, utilizamos o código

---
`producer`
```python

exchange_name

# Define exchange name
exchange_name = 'logs'

# Declare exchange
channel.exchange_declare(
    exchange=exchange_name,
    exchange_type='fanout'
)

# Publish message
channel.basic_publish(
    exchange=exchange_name,
    routing_key='',
    body=message
)
```
---

Para listar os exchanges existentes, use o comando

---
`terminal`
```bash
sudo rabbitmqctl list_exchanges
```
---

**Perguntas** 
1. Quais as diferenças entre o código acima e o produtor da última aula? 
2. O que as `''` significam?
3. Como conectar o consumidor ao servidor, se a fila não existe?

## Temporary queues

Como nosso sistema de logs deve enviar mensagens para quem quiser recebe-las, independentemente do número de consumidores, o produtor não está vinculado a nenhuma fila específica. 

Contudo, o consumidor precisar se conectar ao servidor por meio de uma fila! Esse conexão é feita através de filas temporárias.


As `''` indicam que o valor padrão deve ser usado, tanto no caso do exchange como das filas e, no caso das filas, o uso das aspas permite que o servidor crie um nome aleatório para elas. 

Para garantir a independência de nosso código com qualquer fila que possa ser criada, marcaremos essa fila como `exclusiva`, ou seja, ela deverá ser deletada uma vez que a conexão com o `consumidor` for encerrada!

O código deverá ser alterado para

---
`consumer`
```python
# Declare a queue to consume messages from
result = channel.queue_declare(queue='', exclusive=True)
```
---

O nome aleatório, gerado por esse código, se encontra armazenado em `result.method.queue` e, dessa forma, o nome de nossa fila pode ser definido e acessado por

---
```python
# Define queue name
queue_name = result.method.queue
```
---

Mais informações sobre [filas](https://www.rabbitmq.com/queues.html).

**Perguntas**

1. É suficiente apenas declarar a fila do lado do consumidor?
2. Caso negativo, como associar a fila ao exchange?

## Bindings

* São as relações entre *queues* e *exchanges*.
* Em outras palavras: a *queue* está interessada em mensagens desse *exchange*. 
* Em português, ligações.

![bindings](https://s3-sa-east-1.amazonaws.com/lcpi/12c70e98-c374-45ad-93a3-b8ecffac82bd.png)

São declaradas no consumidor, por:

---
`consumer`
```python
# Declare queue bindings
channel.queue_bind(
    exchange='logs',
    queue=result.method.queue
)
```
---



E para consultar as ligações ativas

---
`terminal`
```bash
sudo rabbitmqctl list_bindings
```
---

**Exercício**
---

Faça o broadcast de mensagens 100_000 mensagens de log para $N$ consumidores interessados. Envie mensagens com intervalos de 1 s.

![python-three-overall](https://s3-sa-east-1.amazonaws.com/lcpi/3db7ed3c-89e5-419d-a6f5-fa495a28caa0.png)

1. Altere o producer da aula passada apropriadamente.
2. Altere o consumer da aula passada para receber do *exchange*.
3. Ligue o producer.
4. Ligue um consumer.

**Pergunta:** O consumer recebeu todas as mensagens esperadas?

5. Ligue mais dois consumers.

**Pergunta:** Os três consumers estão recebendo cópias da mesma mensagem?

6. Desligue um consumer e religue-o em seguida.
   
**Pergunta:** As mensagens foram retomadas de onde o consumer havia sido interrompido? Por quê?


`producer`
---
```python
pass
```
---

`consumer`
---
```python
pass
```
---

**Pergunta**

É possível consumir apenas um subgrupo de mensagens?

# Routing

Nosso sistema de logs acima transmite todas as mensagens para todos os consumidores.

O próximo passo é permitir a filtragem de mensagens com base em sua gravidade (*severity*). 

Por exemplo, podemos definir que nosso script salve em disco apenas os logs críticos e passe adiante as mensagens menos graves.

Primeiramente, vamos adicionar um novo parâmetro à nossa ligação: `routing_key`. Esse parâmetro é definido por um *label* e seu funcionamento depende do *exchange_type*.

No exemplo anterior, usamos `exchange_type = fanout`, caso em que a routing_key é ignorada.

Por exemplo, o script do produtor pode ser alterado da seguinte maneira

---
`consumner`

```python
# Declare queue bindings
channel.queue_bind(
    exchange=exchange_name,
    queue=queue_name,
    routing_key='black'
)
```
---

### Direct exchange

* Manda mensagens para filas em que `routing_key = biding_key`.
* Mensagens que não satisfazem esse critério são descartadas!

![direct-exchange](https://s3-sa-east-1.amazonaws.com/lcpi/92c49b1b-7e19-451a-999d-367996ffc485.png)

**Pergunta**

Explique o diagrama acima.

`producer`

---
```python
# Define exchange name
exchange_name = 'direct_logs'

# Declare exchange
channel.exchange_declare(
    exchange=exchange_name,
    exchange_type='direct'
)

channel.basic_publish(
    exchange=exchange_name,
    routing_key=severity,
    body=message
)
```
---

### Multiple bindings

* É possível ligar diferentes filas com a mesma *routing_key*.
* O *exchange* encaminhará a mesma mensagem para todas elas.

![direct-exchange-multiple](https://s3-sa-east-1.amazonaws.com/lcpi/d12cb388-593c-4f47-a853-92d272195504.png)

**Exercício**
---

Faça o broadcast de 100_000 mensagens de log para $N$ consumidores interessados. 

Divida as mensagens em três grupos, baseados na prioridade do chamado: 
* error: prioridade 1 (`.`) ou 2 (`..`)
* warning: prioridade 3 (`...`) ou 4()`....`)
* info: others

Consuma em duas filas (veja o diagrama abaixo):
* uma deve receber apenas mensagens de erro e salvá-las em disco.  
* a outra deve receber todas as mensagens.

![python-four](https://s3-sa-east-1.amazonaws.com/lcpi/12c71ef5-d786-4dc1-baae-33d1a79e8a64.png)


**Pergunta:** Quantos scripts de producer e consumer são necessários? 1 Producer e 2 Consumers

1. Altere o producer anterior apropriadamente.
2. Altere o consumer anterior apropriadamente.
3. Ligue o producer.
4. Ligue um consumer.

**Pergunta:** O mesmo consumer lida com as duas filas? Não, 1 consumer para cada fila

5. Ligue o segundo consumer.

**Pergunta:** Ele apresenta o mesmo comportamento do primeiro? Não pois recebeu todas as mensagens

**Pergunta:** Quais as informações que o produtor precisa saber sobre o consumidor? Nenhuma

**Pergunta:** Quais as informações que o consumidor precisa saber sobre o produtor? O nome do exchange e das routing_keys



`producer`
---

```python
pass
```

`consumer`
---

```python
pass
```

**Pergunta:** Seria possível flexibilizar ainda mais o roteamento de mensagens e usar padrões para decidir quais mensagens receber? 

# Topics

* Permitem roteamento com critérios múltiplos

## Topic exchange

A *routing_key* apresenta um padrão específico, composto por `strings` e `.`. Exemplos:
* stock.usd.nyse
* quick.orange.rabbit
* frutas.bananas
* etc

A regra de composição de nomes tem dois coringas (*wildcards*):
* `*`: substitui exatamente uma palavra
* `#`: substitui zero ou mais palavras


Exemplo gráfico:

![python-five](https://s3-sa-east-1.amazonaws.com/lcpi/2c9dfa2e-88ce-4a5a-8636-dc420217642e.png)

**Pergunta:** Explique o diagrama acima!

**Pergunta:** Em qual fila serão entregues as mensagens com tópicos
1. quick.orange.rabbit 
2. lazy.orange.elephant
3. quick.orange.fox
4. lazy.brown.fox
5. lazy.pink.rabbit
6. quick.brown.fox
7. orange
8. lazy.orange.new.rabbit

**Exercício**

Supondo que nosso `produtor` envie mensagens para tópicos no padrão `<facility>.<severity>`, como devem ser as rotas dos consumidores para receberem:

1. Todos os logs?
2. Todos os logs do *facility* `kern`?
3. Apenas logs com *severity* `critical`?
4. Logs dos dois itens acima?
5. Em quais filas a mensgaem de exemplo do produtor abaixo será entregue?

Use os produtores e consumidores abaixo para testar suas repostas! 

Exemplos de uso dos códigos:


---
`producer`
```python
python emit_log_topic.py "kern.critical" "A critical kernel error"
```
---

---
`consumer`
```python
python receive_logs_topic.py "#"
```
---



`producer: emit_log_topic.py`
---
```python
import pika
import sys

connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.exchange_declare(exchange='topic_logs', exchange_type='topic')

routing_key = sys.argv[1] if len(sys.argv) > 2 else 'anonymous.info'
message = ' '.join(sys.argv[2:]) or 'Hello World!'
channel.basic_publish(
    exchange='topic_logs', routing_key=routing_key, body=message)
print(" [x] Sent %r:%r" % (routing_key, message))
connection.close()
```
---



`consumer: receive_logs_topic.py`
---
```python
import pika
import sys

connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.exchange_declare(exchange='topic_logs', exchange_type='topic')

result = channel.queue_declare('', exclusive=True)
queue_name = result.method.queue

binding_keys = sys.argv[1:]
if not binding_keys:
    sys.stderr.write("Usage: %s [binding_key]...\n" % sys.argv[0])
    sys.exit(1)

for binding_key in binding_keys:
    channel.queue_bind(
        exchange='topic_logs', queue=queue_name, routing_key=binding_key)

print(' [*] Waiting for logs. To exit press CTRL+C')


def callback(ch, method, properties, body):
    print(" [x] %r:%r" % (method.routing_key, body))


channel.basic_consume(
    queue=queue_name, on_message_callback=callback, auto_ack=True)

channel.start_consuming()
```
---

In [2]:
import random
import datetime as dt

time_stamp = dt.datetime.strftime(dt.datetime.now(), format='%Y-%m-%d %H:%M:%S.%f')
priority = '.' * random.randint(1, 10)
    
# NOVA TAREFA
message = f'{time_stamp} {priority}'

print(len(priority))

print (message)

8
2023-04-28 14:25:36.656717 ........


Para mais informações, consulte a [documentação](https://www.rabbitmq.com/tutorials/tutorial-five-python.html).